In [1]:
import os
#-------------------------- set gpu using tf ---------------------------
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
#-------------------  start importing keras module ---------------------
import keras.backend.tensorflow_backend as K
K.set_session(session)

from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
from keras.utils.training_utils import multi_gpu_model
from keras import optimizers
from keras import regularizers
from multiprocessing import cpu_count

Using TensorFlow backend.


In [2]:
nb_classes = 2
nb_cpus = 4
nb_gpus = 2

image_size = (299, 299)
input_shape= (299, 299, 3)

train_path = "/home/hdd0/Develop/immune/ext-data/data299/train"
valid_path = "/home/hdd0/Develop/immune/ext-data/data299/valid"

In [3]:
with tf.device('/cpu:0'):
    input_tensor = Input(input_shape)
    x = Lambda(xception.preprocess_input)(input_tensor)

    base_model = Xception(input_tensor=x, weights='imagenet', include_top=False)
    m_out = base_model.output
    p_out = GlobalAveragePooling2D()(m_out)
    p_out = Dropout(0.5)(p_out)
    predictions = Dense(nb_classes, activation='softmax', name="predictions")(p_out)

#     for layer in base_model.layers:
#         layer.trainable = False

    model = Model(inputs=base_model.input, outputs=predictions)
    
#     model.load_weights("Xception_first_train.h5")

if nb_gpus > 1:
    parallel_model = multi_gpu_model(model, gpus=nb_gpus)
else:
    parallel_model = model
    
# optimizer = optimizers.SGD(lr=0.01, momentum=0.9, decay=0.0003)
parallel_model.compile(optimizer="Adadelta", 
                       loss='categorical_crossentropy', 
                       metrics=['accuracy'])

# model.summary()

In [4]:
batch_size = 32
epochs = 1

#### custom generator

In [5]:
from generator import DataGenerator

train_gen = DataGenerator(train_path, workers=nb_cpus, target_size=image_size, batch_size=batch_size, shuffle=True)
valid_gen = DataGenerator(valid_path, workers=nb_cpus, target_size=image_size, batch_size=batch_size, shuffle=True)

# files:  523012
# files:  58112


In [ ]:
parallel_model.fit_generator(generator=train_gen, 
                             steps_per_epoch=len(train_gen), 
                             epochs=epochs, 
                             verbose=1,
                             validation_data=valid_gen, 
                             validation_steps=len(valid_gen))

Epoch 1/1
  379/16345 [..............................] - ETA: 2:43:55 - loss: 0.6464 - acc: 0.6187

#### keras generator

In [5]:
img_gen_t = ImageDataGenerator()
train_gen = img_gen_t.flow_from_directory(train_path, target_size=image_size, shuffle=True, batch_size=batch_size)

img_gen_v = ImageDataGenerator()
valid_gen = img_gen_v.flow_from_directory(valid_path, target_size=image_size, shuffle=True, batch_size=batch_size)

Found 523012 images belonging to 2 classes.
Found 58112 images belonging to 2 classes.


In [ ]:
parallel_model.fit_generator(generator=train_gen, 
                             steps_per_epoch=len(train_gen), 
                             epochs=epochs, 
                             verbose=1,
                             validation_data=valid_gen, 
                             validation_steps=len(valid_gen), 
                             workers=nb_cpus, 
                             use_multiprocessing=True, 
                             )

Epoch 1/1
  376/16345 [..............................] - ETA: 2:42:35 - loss: 0.6490 - acc: 0.6138